In [ ]:
# Just a reminder on population for letterboxd

In [4]:
# Extract info username

user = "trschwab"

BASE_URL = "https://letterboxd.com"

In [5]:
def get_diary(a_user):
    return f"{BASE_URL}/{a_user}/films/diary/"

In [6]:
diary_url = get_diary(user)

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [8]:
import pandas as pd
print(pd.__version__)

2.0.1


In [9]:
r = requests.get(diary_url)
soup = BeautifulSoup(r.content)
table = soup.find_all('table')
df = pd.read_html(str(table), extract_links="all")[0]

In [21]:
df

,"(Month, None)","(Day, None)","(Film, None)","(Released, None)","(Rating, None)","(Like, None)","(Rewatch, None)","(Review, None)","(EditYou, None)"
0,"(Apr 2023, /trschwab/films/diary/for/2023/04/)","(30, /trschwab/films/diary/for/2023/04/30/)","(John Mulaney: Baby J, /trschwab/film/john-mul...","(2023, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)"
1,"(, None)","(22, /trschwab/films/diary/for/2023/04/22/)","(Gone Girl, /trschwab/film/gone-girl/)","(2014, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)"
2,"(Mar 2023, /trschwab/films/diary/for/2023/03/)","(19, /trschwab/films/diary/for/2023/03/19/)","(The Devil Wears Prada, /trschwab/film/the-dev...","(2006, None)","(× ★★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)"
3,"(, None)","(03, /trschwab/films/diary/for/2023/03/03/)","(Stalker, /trschwab/film/stalker/1/)","(1979, None)","(× ★★★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/stalker/1/)","(Edit this entry, #)"
4,"(Feb 2023, /trschwab/films/diary/for/2023/02/)","(26, /trschwab/films/diary/for/2023/02/26/)",(Pirates of the Caribbean: The Curse of the Bl...,"(2003, None)","(× ★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/pirates-of-th...","(Edit this entry, #)"
5,"(, None)","(20, /trschwab/films/diary/for/2023/02/20/)","(Rent, /trschwab/film/rent/)","(2005, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)"
6,"(, None)","(18, /trschwab/films/diary/for/2023/02/18/)","(Three Colours: Blue, /trschwab/film/three-col...","(1993, None)","(× ★★★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/three-colours...","(Edit this entry, #)"
7,"(, None)","(15, /trschwab/films/diary/for/2023/02/15/)","(Valentine's Day, /trschwab/film/valentines-day/)","(2010, None)","(× ★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/valentines-day/)","(Edit this entry, #)"
8,"(Jan 2023, /trschwab/films/diary/for/2023/01/)","(29, /trschwab/films/diary/for/2023/01/29/)","(The Fabelmans, /trschwab/film/the-fabelmans/)","(2022, None)","(× ★★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)"
9,"(, None)","(22, /trschwab/films/diary/for/2023/01/22/)","(Snowpiercer, /trschwab/film/snowpiercer/)","(2013, None)","(× ★★★½, #)","(, None)","(, None)","(, None)","(Edit this entry, #)"


In [33]:
username = "trschwab"
for col in df[('Film', None)].unique():
    print(f"https://letterboxd.com/film/{col[1].split('/')[3]}/")

https://letterboxd.com/film/homework-1989
https://letterboxd.com/film/opening-night
https://letterboxd.com/film/john-mulaney-baby-j
https://letterboxd.com/film/gone-girl
https://letterboxd.com/film/the-devil-wears-prada
https://letterboxd.com/film/stalker
https://letterboxd.com/film/pirates-of-the-caribbean-the-curse-of-the-black-pearl
https://letterboxd.com/film/rent
https://letterboxd.com/film/three-colours-blue
https://letterboxd.com/film/valentines-day
https://letterboxd.com/film/the-fabelmans
https://letterboxd.com/film/snowpiercer
https://letterboxd.com/film/the-menu-2022
https://letterboxd.com/film/under-the-silver-lake
https://letterboxd.com/film/8-half
https://letterboxd.com/film/set-it-up
https://letterboxd.com/film/notting-hill
https://letterboxd.com/film/prey-2022
https://letterboxd.com/film/glass-onion
https://letterboxd.com/film/ferris-buellers-day-off
https://letterboxd.com/film/home-alone
https://letterboxd.com/film/planes-trains-and-automobiles
https://letterboxd.com/f

In [8]:
soup

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="en" class="ie6 lte9 lte8 lte7 lte6 no-js"> <![endif]--><!--[if IE 7 ]>    <html lang="en" class="ie7 lte9 lte8 lte7 no-js"> <![endif]--><!--[if IE 8 ]>    <html lang="en" class="ie8 lte9 lte8 no-js"> <![endif]--><!--[if IE 9 ]>    <html lang="en" class="ie9 lte9 no-js"> <![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class="no-mobile no-js" id="html" lang="en"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=1024" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="Troy Schwab’s film diary" name="description"/>
<meta content="https://letterboxd.com/trschwab/films/diary/" property="og:url"/>
<meta content="Troy Schwab’s film diary" property="og:title"/>
<meta content="Troy Schwab’s film diary" property="og:description"/>
<meta content="https://s.ltrbxd.com/static/img/default-share.e38c5d62.png" property="og:image"/>
<meta content="Letterboxd" name="application-name"/>
<meta c

In [10]:
table = soup.find_all('table')
table

[<table cellpadding="0" cellspacing="0" class="table film-table" id="diary-table">
 <thead>
 <tr>
 <th class="th-month" scope="col">Month</th>
 <th class="th-day center" scope="col">Day</th>
 <th class="th-film" scope="col">Film</th>
 <th class="th-released center" scope="col">Released</th>
 <th class="th-rating" scope="col">Rating</th>
 <th class="th-like center" scope="col">Like</th>
 <th class="th-rewatch center" scope="col">Rewatch</th>
 <th class="th-review center" scope="col">Review</th>
 <th class="th-actions film-actions center hide-when-logged-out" data-owner="trschwab" scope="col"><span class="show-for-owner" data-owner="trschwab">Edit</span><span class="hide-for-owner" data-owner="trschwab">You</span></th>
 </tr>
 </thead>
 <tbody>
 <tr class="diary-entry-row not-rated" data-viewing-id="383560722">
 <td class="td-calendar"> <div class="date"> <strong><a href="/trschwab/films/diary/for/2023/04/">Apr</a></strong> <a href="/trschwab/films/diary/for/2023/"><small>2023</small></a

In [13]:
df = pd.read_html(str(table))

In [14]:
df

[       Month  Day                                               Film  \
 0   Apr 2023   30                               John Mulaney: Baby J   
 1        NaN   22                                          Gone Girl   
 2   Mar 2023   19                              The Devil Wears Prada   
 3        NaN    3                                            Stalker   
 4   Feb 2023   26  Pirates of the Caribbean: The Curse of the Bla...   
 5        NaN   20                                               Rent   
 6        NaN   18                                Three Colours: Blue   
 7        NaN   15                                    Valentine's Day   
 8   Jan 2023   29                                      The Fabelmans   
 9        NaN   22                                        Snowpiercer   
 10       NaN   18                                           The Menu   
 11       NaN   16                              Under the Silver Lake   
 12       NaN   12                                 

In [ ]:
df

In [3]:
def get_diary_page_info(a_page):
    '''
    given a diary page, return the info as a df
    '''
    # Get the HTML info
    r = requests.get(diary_url)
    soup = BeautifulSoup(r.content)

    # Extract the table as a df
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]

    return df

In [289]:
def gen_film_url(a_str):
    return f"https://letterboxd.com/film/{a_str[1].split('/')[3]}/"


def get_user_diary_info(a_user):
    '''
    returns a DF of a user's diary information

    # TOOD need to be able to handle users that do not exist
    '''
    df_list = []
    # Get diary URL
    diary_url = get_diary(a_user)
    while diary_url:
        # print(diary_url)
        r = requests.get(diary_url)
        soup = BeautifulSoup(r.content)

        # Extract the table as a df
        table = soup.find_all('table')
        df_list.append(pd.read_html(str(table), extract_links="all")[0])

        # Find out if there are any subsequent pages
        older_link = soup.find_all("a", text="Older")

        if older_link:
            new_page = older_link[0]['href']
            diary_url = f"{BASE_URL}{new_page}"
        else:
            diary_url = False
    return_df = pd.concat(df_list)

    return_df["film_url"] = return_df[('Film', None)].apply(lambda x: gen_film_url(x))

    return return_df

In [36]:
x = get_user_diary_info("trschwab")

In [37]:
x

,"(Month, None)","(Day, None)","(Film, None)","(Released, None)","(Rating, None)","(Like, None)","(Rewatch, None)","(Review, None)","(EditYou, None)",film_url
0,"(May 2023, /trschwab/films/diary/for/2023/05/)","(14, /trschwab/films/diary/for/2023/05/14/)","(Homework, /trschwab/film/homework-1989/)","(1989, None)","(× ★★★½, #)","(, None)","(, None)","(Read the review, /trschwab/film/homework-1989/)","(Edit this entry, #)",https://letterboxd.com/film/homework-1989/
1,"(, None)","(14, /trschwab/films/diary/for/2023/05/14/)","(Opening Night, /trschwab/film/opening-night/)","(1977, None)","(× ★★★★½, #)","(, None)","(, None)","(Read the review, /trschwab/film/opening-night/)","(Edit this entry, #)",https://letterboxd.com/film/opening-night/
2,"(Apr 2023, /trschwab/films/diary/for/2023/04/)","(30, /trschwab/films/diary/for/2023/04/30/)","(John Mulaney: Baby J, /trschwab/film/john-mul...","(2023, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/john-mulaney-baby-j/
3,"(, None)","(22, /trschwab/films/diary/for/2023/04/22/)","(Gone Girl, /trschwab/film/gone-girl/)","(2014, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/gone-girl/
4,"(Mar 2023, /trschwab/films/diary/for/2023/03/)","(19, /trschwab/films/diary/for/2023/03/19/)","(The Devil Wears Prada, /trschwab/film/the-dev...","(2006, None)","(× ★★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/the-devil-wears-pr...
...,...,...,...,...,...,...,...,...,...,...
46,"(, None)","(12, /trschwab/films/diary/for/2020/01/12/)","(The Wind Rises, /trschwab/film/the-wind-rises/)","(2013, None)","(× ★★★★½, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/the-wind-rises/
47,"(, None)","(11, /trschwab/films/diary/for/2020/01/11/)","(Badlands, /trschwab/film/badlands/)","(1973, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/badlands/
48,"(, None)","(09, /trschwab/films/diary/for/2020/01/09/)","(Chef, /trschwab/film/chef/)","(2014, None)","(× ★★, #)","(, None)","(, None)","(Read prior review, /trschwab/film/chef/)","(Edit this entry, #)",https://letterboxd.com/film/chef/
49,"(, None)","(07, /trschwab/films/diary/for/2020/01/07/)","(Wet Hot American Summer, /trschwab/film/wet-h...","(2001, None)","(× ★★★½, #)","(, None)","(, None)","(Read prior review, /trschwab/film/wet-hot-ame...","(Edit this entry, #)",https://letterboxd.com/film/wet-hot-american-s...


In [8]:
x.to_csv("test.csv")

In [30]:
df = x

In [ ]:
# given a film link, can we generate a row of data for it?

In [38]:
f_url = "https://letterboxd.com/film/homework-1989/"

In [66]:
r = requests.get(f_url)
soup = BeautifulSoup(r.content)
print(soup)
# # Extract the table as a df
# table = soup.find_all('table')
# df_list.append(pd.read_html(str(table), extract_links="all")[0])

# # Find out if there are any subsequent pages
# older_link = soup.find_all("a", text="Older")

# if older_link:
#     new_page = older_link[0]['href']
#     diary_url = f"{BASE_URL}{new_page}"
# else:
#     diary_url = False

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="en" class="ie6 lte9 lte8 lte7 lte6 no-js"> <![endif]--><!--[if IE 7 ]>    <html lang="en" class="ie7 lte9 lte8 lte7 no-js"> <![endif]--><!--[if IE 8 ]>    <html lang="en" class="ie8 lte9 lte8 no-js"> <![endif]--><!--[if IE 9 ]>    <html lang="en" class="ie9 lte9 no-js"> <![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class="no-mobile no-js" id="html" lang="en"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=1024" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="Young male students at a local Iranian school are asked about their feelings on homework." name="description"/>
<meta content="video.movie" property="og:type"/>
<meta content="https://letterboxd.com/film/homework-1989/" property="og:url"/>
<meta content="Homework (1989)" property="og:title"/>
<meta content="Young male students at a local Iranian school are asked about their feelings on homework." property="og:desc

In [64]:
# This is one way to get nice simple info
r = requests.get(f_url)
soup = BeautifulSoup(r.content)

description = soup.find("meta", attrs={'name':'description'})
print(description["content"])
title = soup.find("meta", property="og:title")
print(title["content"])
img = soup.find("meta", property="og:image")
print(img["content"])
director = soup.find("meta", attrs={'name':"twitter:data1"})
print(director["content"])
avg_rating = soup.find("meta", attrs={'name':"twitter:data2"})
print(avg_rating["content"])

Young male students at a local Iranian school are asked about their feelings on homework.
Homework (1989)
https://a.ltrbxd.com/resized/sm/upload/7b/jf/ro/u2/homework-1200-1200-675-675-crop-000000.jpg?v=0cc019b307
Abbas Kiarostami
3.95 out of 5


In [56]:
x = soup.find_all("meta")
for i in x:
    print(i)

<meta charset="utf-8"/>
<meta content="width=1024" name="viewport"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="Young male students at a local Iranian school are asked about their feelings on homework." name="description"/>
<meta content="video.movie" property="og:type"/>
<meta content="https://letterboxd.com/film/homework-1989/" property="og:url"/>
<meta content="Homework (1989)" property="og:title"/>
<meta content="Young male students at a local Iranian school are asked about their feelings on homework." property="og:description"/>
<meta content="https://a.ltrbxd.com/resized/sm/upload/7b/jf/ro/u2/homework-1200-1200-675-675-crop-000000.jpg?v=0cc019b307" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="675" property="og:image:height"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@letterboxd" name="twitter:site"/>
<meta content="https://letterboxd.com/film/homework-1989/" name="twitter:

[<meta charset="utf-8"/>, <meta content="width=1024" name="viewport"/>, <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>, <meta content="Young male students at a local Iranian school are asked about their feelings on homework." name="description"/>, <meta content="video.movie" property="og:type"/>, <meta content="https://letterboxd.com/film/homework-1989/" property="og:url"/>, <meta content="Homework (1989)" property="og:title"/>, <meta content="Young male students at a local Iranian school are asked about their feelings on homework." property="og:description"/>, <meta content="https://a.ltrbxd.com/resized/sm/upload/7b/jf/ro/u2/homework-1200-1200-675-675-crop-000000.jpg?v=0cc019b307" property="og:image"/>, <meta content="1200" property="og:image:width"/>, <meta content="675" property="og:image:height"/>, <meta content="summary_large_image" name="twitter:card"/>, <meta content="@letterboxd" name="twitter:site"/>, <meta content="https://letterboxd.com/film/homework-1989/" 

In [206]:
import json
import re

r = requests.get(f_url)
soup = BeautifulSoup(r.content)

# What about that /* <![CDATA[ */ line?

s = soup.find("script", attrs={"type": "application/ld+json"})

# print(s.string)

split_str = s.string.split("/* <![CDATA[ */")[-1].split("/* ]]> */")[0]

info = json.loads(split_str)

# data = json.loads(re.search(r".*", s.string))

# print(data)

In [208]:
info["image"]

'https://a.ltrbxd.com/resized/film-poster/1/1/0/0/8/3/110083-homework-0-230-0-345-crop.jpg?v=555d352b2d'

In [203]:
type(test)

dict

In [92]:
user_df = get_user_diary_info("trschwab")

In [93]:
user_df

,"(Month, None)","(Day, None)","(Film, None)","(Released, None)","(Rating, None)","(Like, None)","(Rewatch, None)","(Review, None)","(EditYou, None)",film_url
0,"(May 2023, /trschwab/films/diary/for/2023/05/)","(14, /trschwab/films/diary/for/2023/05/14/)","(Homework, /trschwab/film/homework-1989/)","(1989, None)","(× ★★★½, #)","(, None)","(, None)","(Read the review, /trschwab/film/homework-1989/)","(Edit this entry, #)",https://letterboxd.com/film/homework-1989/
1,"(, None)","(14, /trschwab/films/diary/for/2023/05/14/)","(Opening Night, /trschwab/film/opening-night/)","(1977, None)","(× ★★★★½, #)","(, None)","(, None)","(Read the review, /trschwab/film/opening-night/)","(Edit this entry, #)",https://letterboxd.com/film/opening-night/
2,"(Apr 2023, /trschwab/films/diary/for/2023/04/)","(30, /trschwab/films/diary/for/2023/04/30/)","(John Mulaney: Baby J, /trschwab/film/john-mul...","(2023, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/john-mulaney-baby-j/
3,"(, None)","(22, /trschwab/films/diary/for/2023/04/22/)","(Gone Girl, /trschwab/film/gone-girl/)","(2014, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/gone-girl/
4,"(Mar 2023, /trschwab/films/diary/for/2023/03/)","(19, /trschwab/films/diary/for/2023/03/19/)","(The Devil Wears Prada, /trschwab/film/the-dev...","(2006, None)","(× ★★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/the-devil-wears-pr...
...,...,...,...,...,...,...,...,...,...,...
46,"(, None)","(12, /trschwab/films/diary/for/2020/01/12/)","(The Wind Rises, /trschwab/film/the-wind-rises/)","(2013, None)","(× ★★★★½, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/the-wind-rises/
47,"(, None)","(11, /trschwab/films/diary/for/2020/01/11/)","(Badlands, /trschwab/film/badlands/)","(1973, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/badlands/
48,"(, None)","(09, /trschwab/films/diary/for/2020/01/09/)","(Chef, /trschwab/film/chef/)","(2014, None)","(× ★★, #)","(, None)","(, None)","(Read prior review, /trschwab/film/chef/)","(Edit this entry, #)",https://letterboxd.com/film/chef/
49,"(, None)","(07, /trschwab/films/diary/for/2020/01/07/)","(Wet Hot American Summer, /trschwab/film/wet-h...","(2001, None)","(× ★★★½, #)","(, None)","(, None)","(Read prior review, /trschwab/film/wet-hot-ame...","(Edit this entry, #)",https://letterboxd.com/film/wet-hot-american-s...


In [99]:
user_df["img_url"] = user_df["film_url"].apply(lambda x: add_img(x))

In [213]:
user_df["small_img_url"] = user_df["film_url"].apply(lambda x: add_small_img(x))

'image'


In [98]:
def add_img(x):
    r = requests.get(x)
    soup = BeautifulSoup(r.content)
    img = soup.find("meta", property="og:image")
    return img["content"]

In [212]:
def add_small_img(x):
    try:
        r = requests.get(x)
        soup = BeautifulSoup(r.content)
        s = soup.find("script", attrs={"type": "application/ld+json"})
        split_str = s.string.split("/* <![CDATA[ */")[-1].split("/* ]]> */")[0]
        info = json.loads(split_str)
        return info["image"]
    except Exception as e:
        print(e)
        return None

In [214]:
user_df

,"(Month, None)","(Day, None)","(Film, None)","(Released, None)","(Rating, None)","(Like, None)","(Rewatch, None)","(Review, None)","(EditYou, None)",film_url,img_url,small_img_url
0,"(May 2023, /trschwab/films/diary/for/2023/05/)","(14, /trschwab/films/diary/for/2023/05/14/)","(Homework, /trschwab/film/homework-1989/)","(1989, None)","(× ★★★½, #)","(, None)","(, None)","(Read the review, /trschwab/film/homework-1989/)","(Edit this entry, #)",https://letterboxd.com/film/homework-1989/,https://a.ltrbxd.com/resized/sm/upload/7b/jf/r...,https://a.ltrbxd.com/resized/film-poster/1/1/0...
1,"(, None)","(14, /trschwab/films/diary/for/2023/05/14/)","(Opening Night, /trschwab/film/opening-night/)","(1977, None)","(× ★★★★½, #)","(, None)","(, None)","(Read the review, /trschwab/film/opening-night/)","(Edit this entry, #)",https://letterboxd.com/film/opening-night/,https://a.ltrbxd.com/resized/sm/upload/ar/64/4...,https://a.ltrbxd.com/resized/film-poster/2/9/6...
2,"(Apr 2023, /trschwab/films/diary/for/2023/04/)","(30, /trschwab/films/diary/for/2023/04/30/)","(John Mulaney: Baby J, /trschwab/film/john-mul...","(2023, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/john-mulaney-baby-j/,https://a.ltrbxd.com/resized/sm/upload/il/xr/o...,https://a.ltrbxd.com/resized/film-poster/9/8/5...
3,"(, None)","(22, /trschwab/films/diary/for/2023/04/22/)","(Gone Girl, /trschwab/film/gone-girl/)","(2014, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/gone-girl/,https://a.ltrbxd.com/resized/sm/upload/6g/ja/b...,https://a.ltrbxd.com/resized/film-poster/1/4/9...
4,"(Mar 2023, /trschwab/films/diary/for/2023/03/)","(19, /trschwab/films/diary/for/2023/03/19/)","(The Devil Wears Prada, /trschwab/film/the-dev...","(2006, None)","(× ★★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/the-devil-wears-pr...,https://a.ltrbxd.com/resized/sm/upload/qo/gq/n...,https://a.ltrbxd.com/resized/film-poster/5/1/7...
...,...,...,...,...,...,...,...,...,...,...,...,...
46,"(, None)","(12, /trschwab/films/diary/for/2020/01/12/)","(The Wind Rises, /trschwab/film/the-wind-rises/)","(2013, None)","(× ★★★★½, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/the-wind-rises/,https://a.ltrbxd.com/resized/sm/upload/m2/5e/v...,https://a.ltrbxd.com/resized/film-poster/1/1/2...
47,"(, None)","(11, /trschwab/films/diary/for/2020/01/11/)","(Badlands, /trschwab/film/badlands/)","(1973, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/badlands/,https://a.ltrbxd.com/resized/sm/upload/xb/c4/3...,https://a.ltrbxd.com/resized/film-poster/4/9/8...
48,"(, None)","(09, /trschwab/films/diary/for/2020/01/09/)","(Chef, /trschwab/film/chef/)","(2014, None)","(× ★★, #)","(, None)","(, None)","(Read prior review, /trschwab/film/chef/)","(Edit this entry, #)",https://letterboxd.com/film/chef/,https://a.ltrbxd.com/resized/sm/upload/il/kp/w...,https://a.ltrbxd.com/resized/film-poster/1/5/1...
49,"(, None)","(07, /trschwab/films/diary/for/2020/01/07/)","(Wet Hot American Summer, /trschwab/film/wet-h...","(2001, None)","(× ★★★½, #)","(, None)","(, None)","(Read prior review, /trschwab/film/wet-hot-ame...","(Edit this entry, #)",https://letterboxd.com/film/wet-hot-american-s...,https://a.ltrbxd.com/resized/sm/upload/ix/ec/8...,https://a.ltrbxd.com/resized/film-poster/5/0/4...


In [215]:
user_df.to_csv("look.csv")

In [218]:
head_test = user_df.head(25)

In [162]:
head_test

,"(Month, None)","(Day, None)","(Film, None)","(Released, None)","(Rating, None)","(Like, None)","(Rewatch, None)","(Review, None)","(EditYou, None)",film_url,img_url
0,"(May 2023, /trschwab/films/diary/for/2023/05/)","(14, /trschwab/films/diary/for/2023/05/14/)","(Homework, /trschwab/film/homework-1989/)","(1989, None)","(× ★★★½, #)","(, None)","(, None)","(Read the review, /trschwab/film/homework-1989/)","(Edit this entry, #)",https://letterboxd.com/film/homework-1989/,https://a.ltrbxd.com/resized/sm/upload/7b/jf/r...
1,"(, None)","(14, /trschwab/films/diary/for/2023/05/14/)","(Opening Night, /trschwab/film/opening-night/)","(1977, None)","(× ★★★★½, #)","(, None)","(, None)","(Read the review, /trschwab/film/opening-night/)","(Edit this entry, #)",https://letterboxd.com/film/opening-night/,https://a.ltrbxd.com/resized/sm/upload/ar/64/4...
2,"(Apr 2023, /trschwab/films/diary/for/2023/04/)","(30, /trschwab/films/diary/for/2023/04/30/)","(John Mulaney: Baby J, /trschwab/film/john-mul...","(2023, None)","(×, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/john-mulaney-baby-j/,https://a.ltrbxd.com/resized/sm/upload/il/xr/o...
3,"(, None)","(22, /trschwab/films/diary/for/2023/04/22/)","(Gone Girl, /trschwab/film/gone-girl/)","(2014, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/gone-girl/,https://a.ltrbxd.com/resized/sm/upload/6g/ja/b...
4,"(Mar 2023, /trschwab/films/diary/for/2023/03/)","(19, /trschwab/films/diary/for/2023/03/19/)","(The Devil Wears Prada, /trschwab/film/the-dev...","(2006, None)","(× ★★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/the-devil-wears-pr...,https://a.ltrbxd.com/resized/sm/upload/qo/gq/n...
5,"(, None)","(03, /trschwab/films/diary/for/2023/03/03/)","(Stalker, /trschwab/film/stalker/1/)","(1979, None)","(× ★★★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/stalker/1/)","(Edit this entry, #)",https://letterboxd.com/film/stalker/,https://a.ltrbxd.com/resized/sm/upload/hm/9i/x...
6,"(Feb 2023, /trschwab/films/diary/for/2023/02/)","(26, /trschwab/films/diary/for/2023/02/26/)",(Pirates of the Caribbean: The Curse of the Bl...,"(2003, None)","(× ★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/pirates-of-th...","(Edit this entry, #)",https://letterboxd.com/film/pirates-of-the-car...,https://a.ltrbxd.com/resized/sm/upload/5h/tg/u...
7,"(, None)","(20, /trschwab/films/diary/for/2023/02/20/)","(Rent, /trschwab/film/rent/)","(2005, None)","(× ★★★, #)","(, None)","(, None)","(, None)","(Edit this entry, #)",https://letterboxd.com/film/rent/,https://a.ltrbxd.com/resized/sm/upload/sq/7k/e...
8,"(, None)","(18, /trschwab/films/diary/for/2023/02/18/)","(Three Colours: Blue, /trschwab/film/three-col...","(1993, None)","(× ★★★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/three-colours...","(Edit this entry, #)",https://letterboxd.com/film/three-colours-blue/,https://a.ltrbxd.com/resized/sm/upload/xr/zj/p...
9,"(, None)","(15, /trschwab/films/diary/for/2023/02/15/)","(Valentine's Day, /trschwab/film/valentines-day/)","(2010, None)","(× ★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/valentines-day/)","(Edit this entry, #)",https://letterboxd.com/film/valentines-day/,https://a.ltrbxd.com/resized/sm/upload/o2/3a/l...


In [216]:
from PIL import Image, ImageEnhance
import urllib.request
from io import BytesIO
import numpy as np

In [219]:
a_list = head_test["small_img_url"].unique()

In [220]:
a_list
b_list = np.reshape(a_list, (-1, 5))

In [226]:
new = Image.new("RGBA", (1000,1700))
for row in range(len(b_list)):
    for col in range(len(b_list[row])):
        try:
            response = requests.get(b_list[row][col])
            img = Image.open(BytesIO(response.content))
            img = img.resize((200,340))
            new.paste(img, (col * 200, row * 340))
        except Exception as e:
            print(e)
new.save("img.png")

In [181]:

for count, item in enumerate(b_list):
    print(item)
    response = requests.get(item)
    img = Image.open(BytesIO(response.content))
    img = img.resize((200,200))
    new.paste(img, ((count % 5) * 200, (count % 5) * 200))
    x += 1

https://a.ltrbxd.com/resized/sm/upload/7b/jf/ro/u2/homework-1200-1200-675-675-crop-000000.jpg?v=0cc019b307
https://a.ltrbxd.com/resized/sm/upload/ar/64/4h/cv/yphUoND4Wy6q9UvRworY5lNUOs5y5C_large-1200-1200-675-675-crop-000000.jpg?v=70ee32e626
https://a.ltrbxd.com/resized/sm/upload/il/xr/ow/hf/mulaney-1200-1200-675-675-crop-000000.jpg?v=5e1f9da582
https://a.ltrbxd.com/resized/sm/upload/6g/ja/bv/xf/gone-girl-1200-1200-675-675-crop-000000.jpg?v=19f256dfa8
https://a.ltrbxd.com/resized/sm/upload/qo/gq/n0/gm/the-devil-wears-prada-1200-1200-675-675-crop-000000.jpg?v=f55d284d46
https://a.ltrbxd.com/resized/sm/upload/hm/9i/xs/np/stalker-1200-1200-675-675-crop-000000.jpg?v=cb028d9961
https://a.ltrbxd.com/resized/sm/upload/5h/tg/uo/z7/pirates-black-pearl-50-1200-1200-675-675-crop-000000.jpg?v=c4cea61022
https://a.ltrbxd.com/resized/sm/upload/sq/7k/ef/lt/rent-1200-1200-675-675-crop-000000.jpg?v=c68fb733c2
https://a.ltrbxd.com/resized/sm/upload/xr/zj/pc/7k/blue-1200-1200-675-675-crop-000000.jpg?v=80

In [193]:
new.save("img.png")

array(['https://a.ltrbxd.com/resized/sm/upload/7b/jf/ro/u2/homework-1200-1200-675-675-crop-000000.jpg?v=0cc019b307',
       'https://a.ltrbxd.com/resized/sm/upload/ar/64/4h/cv/yphUoND4Wy6q9UvRworY5lNUOs5y5C_large-1200-1200-675-675-crop-000000.jpg?v=70ee32e626',
       'https://a.ltrbxd.com/resized/sm/upload/il/xr/ow/hf/mulaney-1200-1200-675-675-crop-000000.jpg?v=5e1f9da582',
       'https://a.ltrbxd.com/resized/sm/upload/6g/ja/bv/xf/gone-girl-1200-1200-675-675-crop-000000.jpg?v=19f256dfa8',
       'https://a.ltrbxd.com/resized/sm/upload/qo/gq/n0/gm/the-devil-wears-prada-1200-1200-675-675-crop-000000.jpg?v=f55d284d46',
       'https://a.ltrbxd.com/resized/sm/upload/hm/9i/xs/np/stalker-1200-1200-675-675-crop-000000.jpg?v=cb028d9961',
       'https://a.ltrbxd.com/resized/sm/upload/5h/tg/uo/z7/pirates-black-pearl-50-1200-1200-675-675-crop-000000.jpg?v=c4cea61022',
       'https://a.ltrbxd.com/resized/sm/upload/sq/7k/ef/lt/rent-1200-1200-675-675-crop-000000.jpg?v=c68fb733c2',
       'https:

In [299]:
# Generate topster for user

user_df = get_user_diary_info("_jelvin")

c_names = ["month", "day", "film", "released", "rating", "like", "rewatch", "review", "edityou", "film_url"]  # , "img_url", "small_img_url"]

for i in range(len(c_names)):
    user_df.rename(columns={ user_df.columns[i]: c_names[i] }, inplace=True)

# Sort by rating
user_df = user_df.sort_values("rating", ascending=False)

user_df = user_df.head(25)

user_df["img_url"] = user_df["film_url"].apply(lambda x: add_img(x))
user_df["small_img_url"] = user_df["film_url"].apply(lambda x: add_small_img(x))

head_test = user_df

a_list = list(head_test["small_img_url"])

b_list = np.reshape(a_list, (-1, 5))

new = Image.new("RGBA", (1150,1725))
for row in range(len(b_list)):
    for col in range(len(b_list[row])):
        try:
            response = requests.get(b_list[row][col])
            img = Image.open(BytesIO(response.content))
            img = img.resize((230,345))
            new.paste(img, (col * 230, row * 345))
        except Exception as e:
            print(e)
new.save("_jelvin_top.png")

In [267]:
user_df

c_names = ["month", "day", "film", "released", "rating", "like", "rewatch", "review", "edityou", "film_url", "img_url", "small_img_url"]

for i in range(len(c_names)):
    print(i)
    user_df.rename(columns={ user_df.columns[i]: c_names[i] }, inplace=True)

0
1
2
3
4
5
6
7
8
9
10
11


In [270]:
user_df["rating"].unique()

array([('× ★★★½', '#'), ('× ★★★★½', '#'), ('×', '#'), ('× ★★★', '#'),
       ('× ★★★★', '#'), ('× ★★★★★', '#'), ('× ★★', '#'), ('× ★★½', '#'),
       ('× ★½', '#'), ('× ★', '#')], dtype=object)

In [271]:
r_stuff = [('× ★★★½', '#'), ('× ★★★★½', '#'), ('×', '#'), ('× ★★★', '#'),
       ('× ★★★★', '#'), ('× ★★★★★', '#'), ('× ★★', '#'), ('× ★★½', '#'),
       ('× ★½', '#'), ('× ★', '#')]

In [272]:
r_stuff

[('× ★★★½', '#'),
 ('× ★★★★½', '#'),
 ('×', '#'),
 ('× ★★★', '#'),
 ('× ★★★★', '#'),
 ('× ★★★★★', '#'),
 ('× ★★', '#'),
 ('× ★★½', '#'),
 ('× ★½', '#'),
 ('× ★', '#')]

In [274]:
ord("★")

9733

In [275]:
ord("½")

189

In [232]:
# So what's all the info we can get from our movie site?

r = requests.get(f_url)
soup = BeautifulSoup(r.content)

# What about that /* <![CDATA[ */ line?
s = soup.find("script", attrs={"type": "application/ld+json"})

split_str = s.string.split("/* <![CDATA[ */")[-1].split("/* ]]> */")[0]
info = json.loads(split_str)

In [233]:
info

{'image': 'https://a.ltrbxd.com/resized/film-poster/1/1/0/0/8/3/110083-homework-0-230-0-345-crop.jpg?v=555d352b2d',
 '@type': 'Movie',
 'director': [{'@type': 'Person',
   'name': 'Abbas Kiarostami',
   'sameAs': '/director/abbas-kiarostami/'}],
 'dateModified': '2023-02-24',
 'productionCompany': [{'@type': 'Organization',
   'name': 'Kanoon',
   'sameAs': '/studio/kanoon/'}],
 'releasedEvent': [{'@type': 'PublicationEvent', 'startDate': '1989'}],
 '@context': 'http://schema.org',
 'url': 'https://letterboxd.com/film/homework-1989/',
 'actors': [{'@type': 'Person',
   'name': 'Babak Ahmadpoor',
   'sameAs': '/actor/babak-ahmadpoor/'},
  {'@type': 'Person',
   'name': 'Farhang Akhavan',
   'sameAs': '/actor/farhang-akhavan/'},
  {'@type': 'Person',
   'name': 'Mohammad Reza Nematzadeh',
   'sameAs': '/actor/mohammad-reza-nematzadeh/'},
  {'@type': 'Person',
   'name': 'Abbas Kiarostami',
   'sameAs': '/actor/abbas-kiarostami/'},
  {'@type': 'Person', 'name': 'Iraj Safavi', 'sameAs': '/

In [250]:
df = pd.DataFrame.from_dict(info)

ValueError: All arrays must be of the same length

In [249]:
df

,0
0,image
1,@type
2,director
3,dateModified
4,productionCompany
5,releasedEvent
6,@context
7,url
8,actors
9,dateCreated


In [281]:
# Try to get int instead of stars..

def get_user_diary_info_int(a_user):
    '''
    returns a DF of a user's diary information

    # TOOD need to be able to handle users that do not exist
    '''
    df_list = []
    # Get diary URL
    diary_url = get_diary(a_user)
    while diary_url:
        # print(diary_url)
        r = requests.get(diary_url)
        soup = BeautifulSoup(r.content)

        # Extract the table as a df
        table = soup.find_all('table')
        # print(table)
        # _tags = soup.find_all('a', {'class': "edit-review-button has-icon icon-16 icon-edit"})
        # print(a_tags)

        df_list.append(pd.read_html(str(table), extract_links="all")[0])

        # Find out if there are any subsequent pages
        older_link = soup.find_all("a", text="Older")

        if older_link:
            new_page = older_link[0]['href']
            diary_url = f"{BASE_URL}{new_page}"
        else:
            diary_url = False
    return_df = pd.concat(df_list)

    return_df["film_url"] = return_df[('Film', None)].apply(lambda x: gen_film_url(x))

    return return_df

print(get_user_diary_info_int("trschwab"))

[<a class="edit-review-button has-icon icon-16 icon-edit" data-contains-spoilers="false" data-delete-viewing-url="/s/viewing:389354618/delete" data-film-id="110083" data-film-name="Homework" data-film-poster="/film/homework-1989/image-150/" data-film-year="1989" data-rating="7" data-review-text="Something about this movie just puts me straight to sleep but I still love it; kiarostami is the man, can’t wait to finish the koker trilogy" data-rewatch="false" data-specified-date="true" data-spoilers-locked="false" data-tags="[  ]" data-viewing-date="2023-05-14" data-viewing-date-str="14 May 2023" data-viewing-id="389354618" href="#" rel="nofollow">Edit this entry</a>, <a class="edit-review-button has-icon icon-16 icon-edit" data-contains-spoilers="false" data-delete-viewing-url="/s/viewing:389354533/delete" data-film-id="29625" data-film-name="Opening Night" data-film-poster="/film/opening-night/image-150/" data-film-year="1977" data-rating="9" data-review-text="This was just such a bad ti

In [287]:
sorted = user_df.sort_values("rating", ascending=False)
sorted.head(1)

,month,day,film,released,rating,like,rewatch,review,edityou,film_url,img_url,small_img_url
24,"(, None)","(01, /trschwab/films/diary/for/2022/12/01/)","(Paris, Texas, /trschwab/film/paris-texas/1/)","(1984, None)","(× ★★★★★, #)","(, None)","(, None)","(Read the review, /trschwab/film/paris-texas/1/)","(Edit this entry, #)",https://letterboxd.com/film/paris-texas/,https://a.ltrbxd.com/resized/sm/upload/4w/w0/y...,https://a.ltrbxd.com/resized/film-poster/5/1/4...
